In [1]:
import pickle
import pandas as pd
from datetime import timedelta
import numpy as np

In [2]:
year=2050
nuc="nuclear_plus"
bati_hyp="ref"
reindus="reindus"
with open("Results_"+str(year)+"_"+nuc+"_"+bati_hyp+"_"+reindus+".pickle", 'rb') as f:
    Variables=pickle.load(f)

In [3]:
print(Variables)

{'power_Dvar':                       TIME    TECHNOLOGIES    power_Dvar
0      2019-04-29 08:00:00         Biomass   1827.900000
1      2019-04-29 08:00:00           Solar   7645.493600
2      2019-04-29 08:00:00     curtailment      0.000000
3      2019-04-29 08:00:00   NewHydroRiver    759.612977
4      2019-04-29 08:00:00             TAC      0.000000
...                    ...             ...           ...
113875 2019-01-27 03:00:00         OldNuke   3937.200000
113876 2019-01-27 03:00:00    WindOffShore  60648.000000
113877 2019-01-27 03:00:00  HydroReservoir      0.000000
113878 2019-01-27 03:00:00            Coal      0.000000
113879 2019-01-27 03:00:00         NewNuke   9028.800000

[113880 rows x 3 columns], 'energy_Pvar':                     TIME   energy_Pvar
0    2019-04-29 08:00:00  72146.429256
1    2019-06-26 10:00:00  90012.103975
2    2019-03-22 11:00:00  93985.843363
3    2019-02-21 06:00:00  69565.098016
4    2019-08-20 10:00:00  71648.959858
...                  ...

# 1. Capacity 

In [4]:
# Production means capacity (MW)
Variables['capacity_Dvar']

,TECHNOLOGIES,capacity_Dvar
0,Biomass,2031.0
1,Solar,100000.0
2,curtailment,0.0
3,NewHydroRiver,1200.0
4,TAC,0.0
5,HydroRiver,11030.0
6,WindOnShore,37000.0
7,CCG,0.0
8,OldNuke,15520.0
9,WindOffShore,80000.0


In [5]:
# Storage means capacity (MW)
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar          141501.545248
dtype: object

In [6]:
Variables['energy_Pvar'].max()

TIME           2019-12-31 23:00:00
energy_Pvar          141501.545248
dtype: object

In [7]:
df=Variables['power_Dvar']
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')]['power_Dvar'].sum())
df2=Variables['storageOut_Pvar']
print(df2[df2.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df3=Variables['storageIn_Pvar']
print(df3[df3.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df4=Variables['stockLevel_Pvar']
print(df4[df4.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(Variables['stockLevel_ini_Dvar'])

                     TIME    TECHNOLOGIES    power_Dvar
61243 2019-12-31 23:00:00         Biomass   2031.000000
61244 2019-12-31 23:00:00           Solar      0.000000
61245 2019-12-31 23:00:00     curtailment      0.000000
61246 2019-12-31 23:00:00   NewHydroRiver    895.799659
61247 2019-12-31 23:00:00             TAC      0.000000
61248 2019-12-31 23:00:00      HydroRiver   8233.891861
61249 2019-12-31 23:00:00     WindOnShore   4432.600000
61250 2019-12-31 23:00:00             CCG      0.000000
61251 2019-12-31 23:00:00         OldNuke  15520.000000
61252 2019-12-31 23:00:00    WindOffShore  17304.000000
61253 2019-12-31 23:00:00  HydroReservoir   8785.000000
61254 2019-12-31 23:00:00            Coal      0.000000
61255 2019-12-31 23:00:00         NewNuke  21710.000000
78912.29151963575
                     TIME      STOCK_TECHNO  storageOut_Pvar
14133 2019-12-31 23:00:00      HydroStorage              0.0
14134 2019-12-31 23:00:00           Battery              0.0
14135 2019-12-3

In [8]:
df=Variables['power_Dvar']
df2=Variables['storageOut_Pvar']
df3=Variables['storageIn_Pvar']
df=df.groupby('TIME')['power_Dvar'].sum()
df2=df2.groupby('TIME')['storageOut_Pvar'].sum()
df3=df3.groupby('TIME')['storageIn_Pvar'].sum()
df_prod=pd.DataFrame()
df_prod['Prod']=df+df2-df3
#df_prod
#df_prod=df_prod.set_index('TIME')

df_diff=pd.DataFrame()
df_diff['diff']=df_prod['Prod']-Variables['total_consumption_Pvar'].set_index('TIME')['total_consumption_Pvar']
print(df_diff['diff'].max())
N_err=0
for i in df_diff.index:
    if np.abs(df_diff.loc[i,'diff'])>1:
        N_err+=1
        print(df_diff.loc[i,'diff'])
print(N_err)

4.3655745685100555e-11
0


In [11]:
df_prod.loc[pd.Timestamp('2019-12-31 23:00:00')]

Prod    78912.29152
Name: 2019-12-31 23:00:00, dtype: float64

In [12]:
Variables['total_consumption_Pvar'].set_index('TIME').loc[pd.Timestamp('2019-12-31 23:00:00')]

total_consumption_Pvar    78912.29152
Name: 2019-12-31 23:00:00, dtype: float64

In [13]:
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar            153146.7491
dtype: object

In [16]:
Variables['total_consumption_Pvar'][Variables['total_consumption_Pvar'].total_consumption_Pvar==Variables['total_consumption_Pvar']['total_consumption_Pvar'].max()]

,TIME,total_consumption_Pvar
1203,2019-01-10 13:00:00,153146.7491


In [9]:
d0=pd.Timestamp('2019-01-01 00:00:00')
d1h=timedelta(hours=1)
df=Variables['flex_consumption_Pvar']
L_EV=[]
for h in range(24):
    L_EV.append(float(df[(df.TIME==d0+h*d1h)&(df.FLEX_CONSUM=='EV')]['flex_consumption_Pvar']))
L_EV

[29929.35293199346,
 21464.0361360506,
 17976.728180410577,
 18475.496747266494,
 20760.91851773587,
 22658.34765789095,
 23416.238072918135,
 4425.99954355268,
 5581.337750877695,
 12706.86362579832,
 13624.119110291438,
 15099.86804763165,
 18004.093045428912,
 14151.765573617662,
 16274.170792973928,
 17205.05533142765,
 16387.451581801648,
 22159.791355977763,
 19544.3739971731,
 18341.039948151323,
 17219.489715064927,
 21504.79427419051,
 35770.65262022089,
 34963.622475400516]